<a href="https://colab.research.google.com/github/kerbezbolatbekkyzy/Simple-Web-Server/blob/main/News.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

NEWS SCRAPPING

In [ ]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time

edge_service = Service(r"C:\Users\asus\Downloads\edgedriver_win64\msedgedriver.exe")

base_url = "https://kaz.nur.kz/latest/"

def main(base_url):
    driver = webdriver.Edge(service=edge_service)
    driver.get(base_url)

    map = {}
    record_id = 0
    seen_articles = set()
    scroll_pause_time = 2
    i=0
    while record_id < 10000:
        # жаңа контект жүктеу үшін беттің соңына дейін жылжыту
        last_height = driver.execute_script("return document.body.scrollHeight")

        while i<200:
            # Беттің төменгі жағына жылжыту
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(scroll_pause_time)
            time.sleep(2)
            i += 1

            # Бет биіктігінің жоғарылағанын тексеру
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break  # егер биіктік өзгермесе тоқтату
            last_height = new_height

        # Беттің HTML мазмұнын аламыз және өңдеуді бастаймыз
        soup = BeautifulSoup(driver.page_source, 'lxml')

        # Жаңалықтар бөлімедерін іздеу
        cards = soup.find_all('article')

        if not cards:
            print("Нет больше данных для извлечения.")
            break

        new_data_found = False
        for card in cards:
            try:
                category = card.find("span", class_="article-preview-category__text").text.strip()
                headline = card.find("h2", class_="article-preview-category__subhead").text.strip().replace('(фото)', '').replace('(видео)', '').strip()
                news_link = card.find("a", class_="article-preview-category__content")['href']

                # Жаңалықтың бірегейлігін тексеру
                article_id = (category, headline)
                if article_id not in seen_articles:
                    seen_articles.add(article_id)
                    record_id += 1
                    map[record_id] = {"Категория": category, "Новости": headline}
                    print(f"Собираем: {category} - {headline}")

                    # Жаңалықтарды жаңа қойындыда ашамыз
                    driver.execute_script("window.open(arguments[0]);", news_link)
                    driver.switch_to.window(driver.window_handles[1])  # жаңа қойындыға өту
                    time.sleep(3)

                    # Жаңалық мазмұнын жинау
                    inner_soup = BeautifulSoup(driver.page_source, 'lxml')
                    formatted_body_content = inner_soup.find('div', class_='formatted-body__content')

                    if formatted_body_content:
                        for figcaption in formatted_body_content.find_all('figcaption'):
                            figcaption.decompose()
                        text = formatted_body_content.get_text(separator=' ', strip=True)
                        text = text.replace('\n', ' ').replace('\u200b', ' ').replace('\xa0', ' ')
                    else:
                        text = "Текст не найден."

                    map[record_id]["Новости"] += " " + text

                    # Жаңалықтар қойындысын жауып, негізгі бетке оралыңыз
                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])
                    new_data_found = True
                    time.sleep(2)
            except Exception as e:
                print(f"Ошибка при обработке карточки: {e}")

        if not new_data_found:
            print("Нет новых данных для извлечения.")
            break

        # Деректерді сақтау
        with open('dataset_news.txt', 'a', encoding='utf-8') as file:
            for key, value in map.items():
                file.write(f'{key}, {value}\n')
            print("Данные сохранены.")

    driver.quit()

main(base_url)


Data Cleaning and Preprocessing
